In [28]:
from itertools import combinations

import pandas as pd
import xmltodict
import networkx as nx

In [2]:
with open("data/eric2020.xml") as fd:
    dict = xmltodict.parse(fd.read())

In [3]:
recs = [rec["metadata"] for rec in dict["records"]["record"]]
df = pd.DataFrame(recs)
df.head()

,dcterms:accessRights,dc:subject,dc:creator,dc:type,eric:keywords_geo,eric:issn,dc:language,dcterms:educationLevel,dc:description,dc:identifier,...,eric:issue,eric:ies_funded,eric:contract_number,eric:wwcguide_link,eric:ies_publication_link,eric:ies_datasource_link,eric:note,eric:wwc_reviewed,eric:keywords_law,eric:keywords_test
0,Yes,"[Community Centers, Student Behavior, Enrichme...","[{'@scheme': 'personal author', '#text': 'Hamm...","[Reports - Research, Tests/Questionnaires]",West Virginia,None,English,Elementary Secondary Education,This evaluation study provides descriptive inf...,"[{'@scheme': 'eric_accno', '#text': 'ED569701'...",...,JAN2021,None,None,None,None,None,None,NaN,NaN,NaN
1,Yes,"[Online Surveys, Program Administration, Admin...","[{'@scheme': 'personal author', '#text': 'Whit...","[Reports - Research, Tests/Questionnaires]",West Virginia,None,English,Elementary Secondary Education,This evaluation study provides descriptive inf...,"[{'@scheme': 'eric_accno', '#text': 'ED569907'...",...,JAN2021,None,None,None,None,None,None,NaN,NaN,NaN
2,Yes,"[Special Education Teachers, Faculty Developme...","[{'@scheme': 'personal author', '#text': 'Stoh...",Reports - Research,West Virginia,None,English,Elementary Secondary Education,The Special Education Technology Integration S...,"[{'@scheme': 'eric_accno', '#text': 'ED569914'...",...,JAN2021,None,None,None,None,None,None,NaN,NaN,NaN
3,Yes,"[Community Colleges, Randomized Controlled Tri...","[{'@scheme': 'personal author', '#text': 'Vish...",Reports - Research,California,None,English,"[Two Year Colleges, Higher Education, Postseco...",Community college students often lack an acade...,"[{'@scheme': 'eric_accno', '#text': 'ED570335'...",...,JAN2021,Yes,EDIES12C0002,https://ies.ed.gov/ncee/wwc/Study/88783,http://ies.ed.gov/ncee/edlabs/projects/project...,None,None,Meets Evidence Standards without Reservations,NaN,NaN
4,Yes,"[Higher Education, Student Financial Aid, Educ...","{'@scheme': 'institution', '#text': 'US Senate...",Legal/Legislative/Regulatory Materials,NaN,None,English,"[Higher Education, Postsecondary Education, Tw...","This hearing of the Committee on Health, Educa...","[{'@scheme': 'eric_accno', '#text': 'ED601246'...",...,JAN2021,None,None,None,None,None,None,NaN,"[Higher Education Act 1965, Stewart B McKinney...",NaN


In [4]:
def extract_authors(dc):
    if type(dc) == list:
        return [
            i.get("#text")
            for i in dc
            if i.get("#text") is not None and i.get("@scheme") != "institution"
        ]
    elif dc.get("#text") is not None and dc.get("@scheme") != "institution":
        return dc.get("#text")


def get_edges(auth_list):
    return list(combinations(auth_list, 2))

In [5]:
df["authors"] = df.apply(lambda row: extract_authors(row["dc:creator"]), axis=1)
df = df[df['authors'].notna()]

In [6]:
df["edges"] = df.apply(lambda row: get_edges(row["authors"]), axis=1)
df.head()

,dcterms:accessRights,dc:subject,dc:creator,dc:type,eric:keywords_geo,eric:issn,dc:language,dcterms:educationLevel,dc:description,dc:identifier,...,eric:contract_number,eric:wwcguide_link,eric:ies_publication_link,eric:ies_datasource_link,eric:note,eric:wwc_reviewed,eric:keywords_law,eric:keywords_test,authors,edges
0,Yes,"[Community Centers, Student Behavior, Enrichme...","[{'@scheme': 'personal author', '#text': 'Hamm...","[Reports - Research, Tests/Questionnaires]",West Virginia,None,English,Elementary Secondary Education,This evaluation study provides descriptive inf...,"[{'@scheme': 'eric_accno', '#text': 'ED569701'...",...,None,None,None,None,None,NaN,NaN,NaN,"[Hammer, Patricia Cahape, White, Larry J.]","[(Hammer, Patricia Cahape, White, Larry J.)]"
1,Yes,"[Online Surveys, Program Administration, Admin...","[{'@scheme': 'personal author', '#text': 'Whit...","[Reports - Research, Tests/Questionnaires]",West Virginia,None,English,Elementary Secondary Education,This evaluation study provides descriptive inf...,"[{'@scheme': 'eric_accno', '#text': 'ED569907'...",...,None,None,None,None,None,NaN,NaN,NaN,"[White, Larry J.]",[]
2,Yes,"[Special Education Teachers, Faculty Developme...","[{'@scheme': 'personal author', '#text': 'Stoh...",Reports - Research,West Virginia,None,English,Elementary Secondary Education,The Special Education Technology Integration S...,"[{'@scheme': 'eric_accno', '#text': 'ED569914'...",...,None,None,None,None,None,NaN,NaN,NaN,"[Stohr, Amber D.]",[]
3,Yes,"[Community Colleges, Randomized Controlled Tri...","[{'@scheme': 'personal author', '#text': 'Vish...",Reports - Research,California,None,English,"[Two Year Colleges, Higher Education, Postseco...",Community college students often lack an acade...,"[{'@scheme': 'eric_accno', '#text': 'ED570335'...",...,EDIES12C0002,https://ies.ed.gov/ncee/wwc/Study/88783,http://ies.ed.gov/ncee/edlabs/projects/project...,None,None,Meets Evidence Standards without Reservations,NaN,NaN,"[Visher, Mary G., Mayer, Alexander K., Johns, ...","[(Visher, Mary G., Mayer, Alexander K.), (Vish..."
5,No,"[Females, Immigrants, College Students, Accult...","[{'@scheme': 'personal author', '#text': 'Lall...",Dissertations/Theses - Doctoral Dissertations,Florida,EISSN-,English,"[Higher Education, Postsecondary Education]",This phenomenological study explored the perce...,"[{'@scheme': 'eric_accno', '#text': 'ED601615'...",...,None,None,None,None,None,NaN,NaN,NaN,"[Lalla, Shireen]",[]


In [17]:
df[['authors']].value_counts

<bound method DataFrame.value_counts of                                                  authors
0             [Hammer, Patricia Cahape, White, Larry J.]
1                                      [White, Larry J.]
2                                      [Stohr, Amber D.]
3      [Visher, Mary G., Mayer, Alexander K., Johns, ...
5                                       [Lalla, Shireen]
...                                                  ...
44893      [Shih, Hui-chia Judy, Huang, Sheng-hui Cindy]
44894                 [Diao, Mingming, Hedberg, John G.]
44895  [Sakiz, Halis, Ekinci, Abdurrahman, Sariçam, H...
44896  [Pantoja, Nancy, Schaeffer, Marjorie W., Rozek...
44897                                 [Hextrum, Kirsten]

[43715 rows x 1 columns]>

In [29]:
list1 = df["authors"].tolist()
list2 = [x for x in list1 if x is not None]  # remove none
node_list = [item for sublist in list2 for item in sublist]
node_list = list(set(node_list))

In [20]:
list1 = df["edges"].tolist()
list2 = [x for x in list1 if x is not None]  # remove none
edge_list = [item for sublist in list2 for item in sublist]

In [ ]:
# df2 = pd.DataFrame({"col": flat_list})
# df2.to_csv("data/test.csv")

In [40]:
G = nx.Graph()
G.add_nodes_from(node_list)
G.add_edges_from(edge_list)

In [58]:
nx.number_connected_components(G)

26173